In [1]:
from importlib import reload
import flymovie as fm
import cnn_models
import cnn_models.siamese_cnn as cn
import cnn_models.evaluate_models as ev
from flymovie.simnuc import Sim
import matplotlib.pyplot as plt
import numpy as np
import os
from fpdf import FPDF
import random
import pickle
from pathlib import Path
import scipy.spatial
import scipy.ndimage as ndi
import tensorflow as tf
from importlib import reload
%load_ext autoreload
%autoreload 2

In [28]:
#---------------------------------------------------------------------------
def make_im_param_inputs(folder, n_repeats=1, mip=False, batch_size=32):
    """
    """
    def get_files_params(subfolder):
        files = os.listdir(subfolder)
        files = [x if x[0] != '.' else None for x in files]
        n_params = len(files[0].split('_')) - 2
        params = np.zeros((0, n_params))
        for f in files:
            p = f.split('_')[1:-1]
            p = [float(x) for x in p]
            p = np.expand_dims(np.array(p), axis=0)
            params = np.vstack((params, p))
        return files, params

    def batch_fetch(ds, batch_size):
        """Optimize dataset for fast parallel retrieval."""
        ds = ds.batch(batch_size, drop_remainder=False)
        ds = ds.prefetch(tf.data.AUTOTUNE)
        return ds

    # Set up input directories.
    cache_dir=folder
    anchor_images_path = cache_dir / "left"
    positive_images_path = cache_dir / "right"
    
    print(get_files_params(anchor_images_path))
    # Create datasets from these sorted files. These are in order and match in pairs.
    """
    anchor_dataset = tf.data.Dataset.from_tensor_slices(im_files)
    
    # Combine datasets to form triplet images, apply shuffle to the whole
    # dataset. So upon iteration, order of triplets will be random, and
    # within triplets, the negative image will shuffle.
    dataset = dataset.shuffle(buffer_size=dataset_full_size)
    dataset = dataset.take(dataset_take_size)

    # Apply preprocessing and rotation via special mappable functions.
    if mip:
        dataset = dataset.map(preprocess_triplets_mip, num_parallel_calls=tf.data.AUTOTUNE)
        if rotate:
            dataset = dataset.map(random_rotate_triplets_mip, num_parallel_calls=tf.data.AUTOTUNE)

    if not mip:
        dataset = dataset.map(preprocess_triplets_3d, num_parallel_calls=tf.data.AUTOTUNE)
        if rotate:
            dataset = dataset.map(random_rotate_triplets_3d, num_parallel_calls=tf.data.AUTOTUNE)
    
    # Divide into training and evaluation, batch and prefetch.
    train_dataset = dataset.take(round(dataset_take_size * 0.8))
    val_dataset = dataset.skip(round(dataset_take_size * 0.8))

    train_dataset = batch_fetch(train_dataset, batch_size)
    val_dataset = batch_fetch(val_dataset, batch_size)

    return train_dataset, val_dataset
    """
make_im_param_inputs(Path('/Users/michaelstadler/Bioinformatics/Projects/rpb1/results/testsims_uPoNivMJ_10'))

(['AaW_8455_13.89_400_771_1_0.41_35_6.05_11.37_rep0.pkl', 'cio_21108_12.58_612_588_2_0.02_35_7.52_16.69_rep0.pkl', 'dXJ_22327_10.58_523_331_1_0.52_39_7.09_10.57_rep0.pkl', 'ciW_7535_13.44_277_215_1_0.55_40_6.86_26.96_rep0.pkl', 'Eer_24451_8.45_630_287_2_0.89_41_4.23_19.98_rep0.pkl', 'DyX_22492_11.67_104_195_2_0.87_49_9.04_17.88_rep0.pkl', 'cOp_16676_12.32_400_893_2_0.73_25_3.94_11.68_rep0.pkl', 'AoB_21007_8.06_739_671_1_0.23_42_1.28_13.87_rep0.pkl', 'DgW_35477_8.46_519_661_1_0.44_39_9.94_20.81_rep0.pkl', 'DoR_14494_11.88_705_617_1_0.72_24_8.68_22.75_rep0.pkl'], array([[8.4550e+03, 1.3890e+01, 4.0000e+02, 7.7100e+02, 1.0000e+00,
        4.1000e-01, 3.5000e+01, 6.0500e+00, 1.1370e+01],
       [2.1108e+04, 1.2580e+01, 6.1200e+02, 5.8800e+02, 2.0000e+00,
        2.0000e-02, 3.5000e+01, 7.5200e+00, 1.6690e+01],
       [2.2327e+04, 1.0580e+01, 5.2300e+02, 3.3100e+02, 1.0000e+00,
        5.2000e-01, 3.9000e+01, 7.0900e+00, 1.0570e+01],
       [7.5350e+03, 1.3440e+01, 2.7700e+02, 2.1500e+02, 1

In [219]:
cache_dir = Path('/Users/michaelstadler/Bioinformatics/Projects/rpb1/results/testsims_samesies')
train, val = cn.make_triplet_inputs(cache_dir, rotate=True)
ev.visualize_batch(train)

Matching triplets...
0


interactive(children=(Dropdown(description='Color', index=9, options=('Reds', 'viridis', 'magma', 'Gators', 'p…

In [208]:
np.random.randint(100_000)

98731

In [205]:
def test(im):
    for i in im:
        i = ndi.rotate(i, 30)
    return im

for batch in train:
    tf.py_function(test,[batch,],[tf.float32,])

2022-03-02 08:48:56.763419: W tensorflow/core/framework/op_kernel.cc:1751] Unknown: KeyboardInterrupt: 
Traceback (most recent call last):

  File "/Users/michaelstadler/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 247, in __call__
    return func(device, token, args)

  File "/Users/michaelstadler/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 135, in __call__
    ret = self._func(*args)

  File "/Users/michaelstadler/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 620, in wrapper
    return func(*args, **kwargs)

  File "/var/folders/46/dftvy0sd0jv10kyqlkqc2d8m0000gn/T/ipykernel_11687/3824729030.py", line 3, in test
    i = ndi.rotate(i, 30)

  File "/Users/michaelstadler/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/scipy/ndimage/interpolation.py", line 955, in rotate
    affine_transform(ia, rot_matrix, offset, out_plane_shape,

  Fi

UnknownError: KeyboardInterrupt: 
Traceback (most recent call last):

  File "/Users/michaelstadler/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 247, in __call__
    return func(device, token, args)

  File "/Users/michaelstadler/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 135, in __call__
    ret = self._func(*args)

  File "/Users/michaelstadler/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 620, in wrapper
    return func(*args, **kwargs)

  File "/var/folders/46/dftvy0sd0jv10kyqlkqc2d8m0000gn/T/ipykernel_11687/3824729030.py", line 3, in test
    i = ndi.rotate(i, 30)

  File "/Users/michaelstadler/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/scipy/ndimage/interpolation.py", line 955, in rotate
    affine_transform(ia, rot_matrix, offset, out_plane_shape,

  File "/Users/michaelstadler/opt/anaconda3/envs/tf2/lib/python3.9/site-pa

In [214]:
import random
import string

nfiles = 10_000
filenames = []
for _ in range(nfiles):
    filename = ''.join([random.choice(string.ascii_letters) for i in range(3)])
    for i in range(8):
        n = random.choice([1,2,3,4,5,6,7,8,9])
        filename = filename + '_' + str(n)
    filename = filename + '_rep0.pkl'
    filenames.append(filename)

In [215]:
files4 = filenames.copy()


In [217]:
_ = cn.match_file_triplets(files3, files4)

Getting param stats
Got param stats
0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500


In [218]:
_ = cn.match_file_triplets(files1, files2)

Getting param stats
Got param stats
0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
54500
55000
55500
56000
56500
57000
57500
58000
58500
59000
59500
60000
60500
61000
61500
62000
62500
63000
63500
64000
64500
65000
65500
66000
66500
67000
67500
68000
68500
69000
69500
70000
70500
71000
71500
72000
72500
73000
73500
74000
74500
75000
75500
76000
76500
77000
77500
78000
78500
79000
79500
80000
80500
81000
81500
8200

In [199]:
np.random.choice(5, 5, replace=False)

array([0, 4, 3, 1, 2])